## Read input data

Here, we consider the pre-processed data to be already available, as provided in the data/ folder in our github repository: https://github.com/anazhaw/tutorial_branch_length/tree/master/data

We have separated the case of direct lineage genes versus subHOGs. In the second case, the expression profile must be averaged over the expression patterns of member genes. 

In [32]:
import pandas as pd 
import ast
from numpy import mean
from numpy import std
from numpy.random import randn
from numpy.random import seed
from matplotlib import pyplot
from scipy import stats
from scipy.stats import pearsonr, kendalltau, spearmanr, ttest_ind, ttest_rel, wilcoxon

## settings for image size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10



In [34]:
## read input data
direct_lineage_genes = pd.read_csv('../data/gene_expr_profile_taxon.csv')

# we need to convert the list of tissues that is read as a string back to a python list
direct_lineage_genes["tissues"] = direct_lineage_genes.apply(lambda row: ast.literal_eval(row["tissues"]), axis=1)

In [35]:
# sample data points: 
## the duplication_hog represents the parent HOG of the paralogous copy "gene"
## the tissues field stores all anatomical entity where the gene is expressed, according to Bgee
## the branch_length stores the length of the branch between "gene" and "duplication_hog"
## the taxon stored the NCBI taxonomic ID of the gene

direct_lineage_genes.head()

,Unnamed: 0,duplication_hog,gene,tissues,branch_length,taxon
0,0,https://omabrowser.org/oma/hog/information/HOG...,https://omabrowser.org/oma/info/HORSE02395,"{testis, gluteus medius, synovial membrane of ...",8.101220,http://purl.uniprot.org/taxonomy/9796
1,1,https://omabrowser.org/oma/hog/information/HOG...,https://omabrowser.org/oma/info/HORSE00374,"{epithelium of bronchus, chorionic villus, tro...",8.747494,http://purl.uniprot.org/taxonomy/9796
2,2,https://omabrowser.org/oma/hog/information/HOG...,https://omabrowser.org/oma/info/MACMU06139,"{testis, kidney, brain, spermatocyte, heart, f...",1.563263,http://purl.uniprot.org/taxonomy/9544
3,3,https://omabrowser.org/oma/hog/information/HOG...,https://omabrowser.org/oma/info/MACMU11463,"{multi-cellular organism, skeletal muscle tiss...",0.376270,http://purl.uniprot.org/taxonomy/9544
4,4,https://omabrowser.org/oma/hog/information/HOG...,https://omabrowser.org/oma/info/MACMU17835,"{testis, kidney, brain, heart, liver, multi-ce...",1.097665,http://purl.uniprot.org/taxonomy/9544


In [36]:
# how many genes do we have?
direct_lineage_genes[["gene"]].count()

gene    14966
dtype: int64

## Case 1. Relation between expression breadth and branch length

The expression breadth is defined as the number of tissues in which a paralogous copy (gene / subHOG) is expressed. 


In [37]:
direct_lineage_genes["expression_breadth"] = direct_lineage_genes.apply(lambda gene_row_info: len(gene_row_info["tissues"]), axis=1)

In [38]:
# checking that the value was extracted correctly...
direct_lineage_genes[["gene", "tissues", "expression_breadth"]].sort_values(by='expression_breadth') #, ascending=False

,gene,tissues,expression_breadth
11061,https://omabrowser.org/oma/info/CAVPO09689,{brain},1
8021,https://omabrowser.org/oma/info/ORNAN18173,{testis},1
8004,https://omabrowser.org/oma/info/ERIEU00360,{prefrontal cortex},1
13097,https://omabrowser.org/oma/info/ANOCA00055,{liver},1
13099,https://omabrowser.org/oma/info/ANOCA00056,{lung},1
13102,https://omabrowser.org/oma/info/RATNO19592,{brain},1
13103,https://omabrowser.org/oma/info/RATNO19587,{liver},1
5508,https://omabrowser.org/oma/info/FELCA08121,{liver},1
7988,https://omabrowser.org/oma/info/ERIEU11346,{adult mammalian kidney},1
7987,https://omabrowser.org/oma/info/ERIEU06575,{adult mammalian kidney},1


In [ ]:
expression_breadth = direct_lineage_genes[["e]]